<a href="https://colab.research.google.com/github/matbutom/maquina-de-contrapropaganda/blob/main/letras_generativas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
# Crea primero el directorio base si no existe
mkdir -p /content/recortes_letras

# Crea los subdirectorios de A a Z
for i in {A..Z}; do
  mkdir -p /content/recortes_letras/$i
done

In [ ]:
%%bash

# 1. Definir el URL base de tu repositorio de GitHub
REPO_URL="https://github.com/matbutom/maquina-de-contrapropaganda.git"
REPO_NAME="maquina-de-contrapropaganda"
TARGET_DIR="/content/recortes_letras"

echo "Clonando el repositorio completo ($REPO_NAME) en el entorno de Colab..."
# Clonar el repositorio
git clone $REPO_URL

# 2. Mover las carpetas con imágenes al directorio de trabajo (recortes_letras)
SOURCE_CONTENT="$REPO_NAME/recortes_letras/*"

echo "Moviendo el contenido de las carpetas de letras (A, B, C...) a $TARGET_DIR..."
# 'cp -r' copia recursivamente el contenido de las subcarpetas A-Z
cp -r $SOURCE_CONTENT $TARGET_DIR/

# 3. Limpiar el repositorio clonado (ya no se necesita)
echo "Limpiando el repositorio clonado..."
rm -rf $REPO_NAME

# 4. Verificación: Mostrar el contenido de la carpeta 'A' para confirmar que las imágenes se cargaron
echo "✅ ¡Carga completa! Verificando la carpeta 'A':"
ls -l $TARGET_DIR/A | head -n 5

In [ ]:
!rm -rf ~/tensorflow_datasets/maquina_contrapropaganda


In [ ]:
# ============================================================
# 🧩 Limpieza y redimensionado físico del dataset
# ============================================================

import os
from PIL import Image

base_dir = "/content/recortes_letras"
target_size = (64, 64)

for root, dirs, files in os.walk(base_dir):
    for f in files:
        if not f.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        path = os.path.join(root, f)
        try:
            im = Image.open(path).convert("RGB")
            im = im.resize(target_size, Image.LANCZOS)
            im.save(path)
        except Exception as e:
            print("⚠️ Error con", path, "→", e)

print("✅ Todas las imágenes fueron redimensionadas físicamente a 64×64 px.")


In [ ]:
# ============================================================
# 🧩 Verificador de dataset — reconstruye solo si hay letras nuevas
# ============================================================

import os
import tensorflow_datasets as tfds

# ruta base donde están las letras (ajústala si usas Drive)
data_dir = '/content/recortes_letras'
builder_dir = os.path.expanduser('~/tensorflow_datasets/maquina_contrapropaganda')

# función auxiliar para listar carpetas válidas
def contar_carpetas(path):
    return sorted([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])

# carpetas actuales detectadas
carpetas_actuales = contar_carpetas(data_dir)
num_actual = len(carpetas_actuales)

# cuántas clases tenía el dataset anterior (si existe)
prev_num = 0
if os.path.exists(builder_dir):
    try:
        info = tfds.builder('maquina_contrapropaganda').info
        prev_num = info.features["label"].num_classes
    except Exception:
        pass

print(f"📦 Letras actuales detectadas: {carpetas_actuales}")
print(f"🧠 Dataset anterior: {prev_num} clases | Nuevo: {num_actual} clases")

# si hay nuevas letras, borrar dataset cacheado
if num_actual > prev_num:
    print("⚠️ Se detectaron nuevas letras. Regenerando dataset completo...")
    !rm -rf ~/tensorflow_datasets/maquina_contrapropaganda
else:
    print("✅ No hay cambios en las clases, se mantiene el dataset anterior.")


In [ ]:
# ============================================================
# 🔍 Verificación física de tamaños reales en disco
# ============================================================

from PIL import Image
import os

base_dir = "/content/recortes_letras"
malas = []

for root, dirs, files in os.walk(base_dir):
    for f in files:
        if not f.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        path = os.path.join(root, f)
        try:
            with Image.open(path) as im:
                if im.size != (64, 64):
                    malas.append((path, im.size))
        except Exception as e:
            malas.append((path, "❌ error"))

print(f"Total de imágenes fuera de tamaño esperado: {len(malas)}")
for i, (p, s) in enumerate(malas[:10]):
    print(f"{i+1:02d}. {p} → {s}")


In [ ]:
# ============================================================
# 📦 Custom Dataset — Máquina de Contrapropaganda
# ============================================================

import tensorflow_datasets as tfds
import tensorflow as tf
import os

_DESCRIPTION = """
Dataset visual para el proyecto 'Máquina de Contrapropaganda'.
Contiene letras recortadas clasificadas por carpeta (A–Z),
extraídas de carteles propagandísticos.
"""

_CITATION = """
@misc{rafita2025maquinacontrapropaganda,
  title={Máquina de Contrapropaganda Dataset},
  author={Arce, Mateo},
  year={2025},
  howpublished={Rafita Studio / Universidad de Chile}
}
"""

class MaquinaContrapropaganda(tfds.core.GeneratorBasedBuilder):
    VERSION = tfds.core.Version('1.0.0')

    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=_DESCRIPTION,
            features=tfds.features.FeaturesDict({
                "image": tfds.features.Image(shape=(None, None, 3)),
                "label": tfds.features.ClassLabel(names=[chr(i) for i in range(65, 91)])  # A–Z
            }),
            supervised_keys=("image", "label"),
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        data_dir = os.path.expanduser('/content/recortes_letras')
        return {"train": self._generate_examples(data_dir)}

    def _generate_examples(self, path):
        for label_name in sorted(os.listdir(path)):
            label_dir = os.path.join(path, label_name)
            if not os.path.isdir(label_dir):
                continue
            for img_name in os.listdir(label_dir):
                if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
                    yield img_name, {
                        "image": os.path.join(label_dir, img_name),
                        "label": label_name,
                    }

# === Construcción del dataset ===
builder = MaquinaContrapropaganda()
builder.download_and_prepare()

ds = builder.as_dataset(split="train", as_supervised=True)

print("✅ Dataset cargado correctamente.")
print("Clases detectadas:", builder.info.features["label"].names)



In [ ]:
# ============================================================
# 👁️ Visualización de ejemplos del dataset
# ============================================================

import matplotlib.pyplot as plt

for image, label in ds.take(9):
    plt.figure(figsize=(2, 2))
    plt.imshow(image)
    plt.title(builder.info.features["label"].int2str(label.numpy()))
    plt.axis("off")
plt.show()


In [ ]:
# ============================================================
# 🛠️ Redimensionado físico forzado (solo las malas)
# ============================================================

from PIL import Image

for path, size in malas:
    try:
        im = Image.open(path).convert("RGB")
        im = im.resize((64, 64), Image.LANCZOS)
        im.save(path)
    except Exception as e:
        print("❌ No se pudo reparar:", path)

print("✅ Todas las imágenes malas fueron corregidas.")


In [ ]:
# ============================================================
# 🧩 División automática del dataset en train / val / test
# ============================================================

import tensorflow as tf
import math

# tamaño total del dataset
total = sum(1 for _ in ds)
train_size = math.floor(total * 0.8)
val_size = math.floor(total * 0.1)
test_size = total - train_size - val_size

print(f"📊 Total de ejemplos: {total}")
print(f"🔹 Train: {train_size} | 🔸 Val: {val_size} | ⚪ Test: {test_size}")

# --- dividir usando el método take() y skip() ---
train_ds = ds.take(train_size)
val_ds = ds.skip(train_size).take(val_size)
test_ds = ds.skip(train_size + val_size)

# --- normalizar imágenes ---
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(img, label):
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img, label

train_ds = train_ds.map(preprocess).cache().shuffle(1000).batch(32).prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().batch(32).prefetch(AUTOTUNE)
test_ds = test_ds.map(preprocess).cache().batch(32).prefetch(AUTOTUNE)

print("✅ Datasets divididos y listos para entrenamiento.")


In [ ]:
# ============================================================
# ✅ Comprobación de tamaño de batch y forma de imágenes
# ============================================================

for imgs, labels in train_ds.take(1):
    print("✅ batch shape:", imgs.shape)
    print("🔹 dtype:", imgs.dtype)
    print("🔹 rango de valores:", tf.reduce_min(imgs).numpy(), "→", tf.reduce_max(imgs).numpy())

    # muestra una de las imágenes para confirmar visualmente
    import matplotlib.pyplot as plt
    plt.imshow(imgs[0])
    plt.title(f"Ejemplo de imagen — tamaño {imgs[0].shape}")
    plt.axis("off")
    plt.show()


In [ ]:
# ============================================================
# 🧩 Configuración general
# ============================================================

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

IMG_SIZE = 64
EPOCHS = 40

# ============================================================
# 🔧 Dataset sin etiquetas y con repetición infinita
# ============================================================

def ensure_valid_image(img):
    # normaliza y redimensiona cada imagen a 64x64
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    return tf.ensure_shape(img, [IMG_SIZE, IMG_SIZE, 3])

train_ds_no_labels = (
    train_ds.unbatch()
    .map(lambda x, y: ensure_valid_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(512)
    .batch(32)
    .repeat()
    .prefetch(tf.data.AUTOTUNE)
)

val_ds_no_labels = (
    val_ds.unbatch()
    .map(lambda x, y: ensure_valid_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .repeat()
    .prefetch(tf.data.AUTOTUNE)
)

print("✅ Datasets verificados:")
for imgs in train_ds_no_labels.take(1):
    print("train batch:", imgs.shape)
for imgs in val_ds_no_labels.take(1):
    print("val batch:", imgs.shape)


# ============================================================
# 🎨 VisualCallback corregido (seguro y estable)
# ============================================================

class VisualCallback(tf.keras.callbacks.Callback):
    def __init__(self, sample_batch, save_dir="/content/outputs", interval=5):
        super().__init__()
        self.sample_batch = sample_batch
        self.save_dir = save_dir
        self.interval = interval
        os.makedirs(save_dir, exist_ok=True)
        self.generated_images = [] # List to store generated images for GIF

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.interval != 0:
            return

        sample_imgs = self.sample_batch[:8]
        z_mean, z_log_var, z = self.model.encoder(sample_imgs)
        reconstructed = self.model.decoder(z)

        n = 8
        fig, axes = plt.subplots(2, n, figsize=(n * 1.5, 3))
        for i in range(n):
            axes[0, i].imshow(sample_imgs[i])
            axes[0, i].axis("off")
            axes[1, i].imshow(reconstructed[i])
            axes[1, i].axis("off")
        plt.tight_layout()

        # Save the figure as an image for later GIF creation
        path = os.path.join(self.save_dir, f"epoch_{epoch+1:03d}.png")
        plt.savefig(path)
        plt.close(fig)
        print(f"🌀 Letras alucinadas guardadas en: {path}")

        # Display the generated images live
        plt.figure(figsize=(n * 1.5, 3))
        for i in range(n):
             plt.subplot(2, n, i + 1)
             plt.imshow(sample_imgs[i])
             plt.axis("off")
             plt.subplot(2, n, i + n + 1)
             plt.imshow(reconstructed[i])
             plt.axis("off")
        plt.suptitle(f"Epoch {epoch+1}", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


        # Store the generated image batch for GIF creation
        self.generated_images.append(reconstructed.numpy())


# ============================================================
# ⚙️ Definición de pérdida del VAE
# ============================================================

# This loss function is no longer directly used by vae.fit because
# we define a custom train_step in the VAE model.
def vae_total_loss(y_true, y_pred):
    reconstruction_loss = tf.reduce_mean(
        tf.keras.losses.binary_crossentropy(y_true, y_pred)
    ) * IMG_SIZE * IMG_SIZE * 3
    # KL divergence loss is calculated in the train_step
    return reconstruction_loss # This will be combined with KL loss in train_step


# # ============================================================
# # 🧠 Entrenamiento del VAE (versión estable) - DEPRECATED
# # ============================================================

# # obtenemos un batch de muestra para el callback
# sample_batch = next(iter(train_ds_no_labels))

# vae = VAE(encoder, decoder)
# vae.compile(optimizer=tf.keras.optimizers.Adam(), loss=vae_total_loss)

# vae.fit(
#     train_ds_no_labels,
#     validation_data=val_ds_no_labels,
#     epochs=EPOCHS,
#     steps_per_epoch=50,
#     validation_steps=10,
#     callbacks=[VisualCallback(sample_batch)],
#     verbose=1
# )


# # ============================================================
# # 💾 Guardado de modelos entrenados - DEPRECATED
# # ============================================================

# decoder.save("/content/drive/MyDrive/maquina-de-contrapropaganda/models/decoder_solo.keras")
# encoder.save("/content/drive/MyDrive/maquina-de-contrapropaganda/models/encoder_solo.keras")
# vae.save("/content/drive/MyDrive/maquina-de-contrapropaganda/models/vae_completo.keras")

# print("✅ Modelos guardados correctamente en Drive.")

In [ ]:
# ============================================================
# ⚙️ Función para preparar el Dataset por Letra (GRIS)
# ============================================================

def prepare_dataset_for_letter(base_ds, target_label_name):
    """
    Filtra, preprocesa y prepara el dataset para entrenar una sola letra en GRIS.
    """
    global IMG_SIZE
    BATCH_SIZE = 32
    AUTOTUNE = tf.data.AUTOTUNE

    target_label_int = builder.info.features["label"].str2int(target_label_name)

    # 1. Filtrar el dataset base
    ds_filtered = base_ds.filter(lambda x, y: tf.equal(y, target_label_int)).map(lambda x, y: x)

    def ensure_valid_image(img):
        # Normalizar a [0, 1]
        img = tf.image.convert_image_dtype(img, tf.float32)
        # 🚨 CAMBIO CLAVE: Conversión a escala de grises
        img = tf.image.rgb_to_grayscale(img)
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        # Aseguramos la forma: [64, 64, 1] (UN canal)
        return tf.ensure_shape(img, [IMG_SIZE, IMG_SIZE, 1])

    ds_final = (
        ds_filtered
        .map(ensure_valid_image, num_parallel_calls=AUTOTUNE)
        .shuffle(512)
        .batch(BATCH_SIZE)
        .repeat()
        .prefetch(AUTOTUNE)
    )

    return ds_final

print("✅ Función prepare_dataset_for_letter actualizada a GRIS (1 canal).")

In [ ]:
# ============================================================
# 🧠 Definición del Encoder (versión estable GRIS)
# ============================================================

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
# Se asumen LATENT_DIM=20, IMG_SIZE=64

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def make_encoder_model():
    # 🚨 CAMBIO CLAVE: Input shape es IMG_SIZE x IMG_SIZE x 1 (GRIS)
    encoder_inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 1))

    # Resto de capas Conv2D... (la cantidad de filtros se mantiene igual)
    x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
    x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)

    z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
    z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
    z = layers.Lambda(sampling, name="z")([z_mean, z_log_var])

    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    return encoder

print("✅ Función make_encoder_model actualizada a GRIS (1 canal de entrada).")

In [ ]:
# ============================================================
# 🧠 Definición del Decoder (versión estable GRIS)
# ============================================================

def make_decoder_model():
    # Decoder network
    latent_inputs = keras.Input(shape=(LATENT_DIM,))
    x = layers.Dense(8 * 8 * 64, activation="relu")(latent_inputs)
    x = layers.Reshape((8, 8, 64))(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x) # 16x16
    x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x) # 32x32

    # 🚨 CAMBIO CLAVE: Capa final con 1 canal de salida
    x = layers.Conv2DTranspose(1, 3, activation="sigmoid", strides=2, padding="same")(x) # 64x64x1

    decoder_outputs = x
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    return decoder

print("✅ Función make_decoder_model actualizada a GRIS (1 canal de salida).")

In [ ]:
# ============================================================
# 🧠 Entrenamiento del VAE (Bucle por Letra)
# ============================================================

# Define las letras que quieres entrenar (A-Z)
LETRAS_A_ENTRENAR = builder.info.features["label"].names # Lista completa A-Z
# Para pruebas, descomenta la línea de abajo:
# LETRAS_A_ENTRENAR = ['A', 'B']

EPOCHS_PER_LETTER = 50
STEPS_PER_EPOCH = 50 # Un valor fijo. Ajusta según el tamaño de tu dataset.

# Rutas de guardado (asegúrate de que tu Drive esté montado si usas esta ruta)
MODEL_DIR = "/content/drive/MyDrive/maquina-de-contrapropaganda/models_por_letra"
os.makedirs(MODEL_DIR, exist_ok=True)
BASE_DATASET = ds

for letter in LETRAS_A_ENTRENAR:
    print(f"\n====================== INICIANDO ENTRENAMIENTO: {letter} ======================")

    # A. Preparar datasets específicos para esta letra
    # Usamos el dataset base 'ds' y lo filtramos por la letra.
    train_ds_letter = prepare_dataset_for_letter(BASE_DATASET, letter)

    # B. Obtener un batch de muestra para el callback de visualización
    # Se obtienen 8 muestras de la letra que se va a entrenar.
    sample_batch = next(iter(train_ds_letter.unbatch().take(8).batch(8)))

    # C. RE-INICIALIZAR MODELOS (CLAVE para que aprenda desde cero cada letra)
    # NOTA: Debes tener las funciones make_encoder_model() y make_decoder_model() definidas.
    #
    try:
        encoder = make_encoder_model() # Función que crea el modelo del Encoder
        decoder = make_decoder_model() # Función que crea el modelo del Decoder
        vae = VAE(encoder, decoder)    # Clase VAE que combina ambos
    except NameError:
        print("❌ ERROR: make_encoder_model, make_decoder_model o VAE no están definidas.")
        print("Por favor, comparte las celdas de la arquitectura del modelo.")
        continue # Salta a la siguiente letra

    # D. Compilar y Entrenar
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=vae_total_loss)

    print(f"🔹 Entrenamiento VAE para la letra '{letter}'...")
    vae.fit(
        train_ds_letter,
        epochs=EPOCHS_PER_LETTER,
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=[VisualCallback(sample_batch, save_dir=f"/content/outputs/{letter}")],
        verbose=1
    )

    # E. Guardar los modelos entrenados
    decoder.save(os.path.join(MODEL_DIR, f"decoder_{letter}.keras"))
    encoder.save(os.path.join(MODEL_DIR, f"encoder_{letter}.keras"))

    print(f"✅ Modelos y salidas para la letra '{letter}' guardados en: {MODEL_DIR}")

In [ ]:
# ============================================================
# ⚙️ Generación de Letras desde el Espacio Latente (CON GUARDADO PNG)
# ============================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# --- Configuraciones Asumidas (las mismas) ---
TEST_LETTER = 'C'
MODEL_DIR = "/content/drive/MyDrive/maquina-de-contrapropaganda/models_por_letra"
LATENT_DIM = 20
N_SAMPLES = 10

# --- Directorio de Salida para los PNGs ---
OUTPUT_PNG_DIR = f"/content/generated_letters/{TEST_LETTER}"
os.makedirs(OUTPUT_PNG_DIR, exist_ok=True)
print(f"Directorio de guardado: {OUTPUT_PNG_DIR}")


# 1. Cargar el modelo Decoder entrenado (usando el código anterior)
# ... (Aquí va tu código de carga del decoder_model y load_weights) ...
# Para que esta celda sea completa, asumo que tienes el código de carga aquí:
try:
    decoder_model = make_decoder_model()
    decoder_path = os.path.join(MODEL_DIR, f"decoder_{TEST_LETTER}.keras")
    decoder_model.load_weights(decoder_path)
    print(f"✅ Decoder cargado exitosamente.")
except Exception as e:
    print(f"❌ ERROR al cargar el Decoder. Asegúrate de que el modelo exista. Error: {e}")
    raise


# 2. Generar muestras aleatorias del Espacio Latente (Z)
z_samples = tf.random.normal(shape=(N_SAMPLES, LATENT_DIM))

# 3. Generar las imágenes
reconstructed_images = decoder_model.predict(z_samples)

print(f"✅ Generación completada. Guardando {N_SAMPLES} imágenes como PNG...")

# 4. BUCLE DE GUARDADO DE PNG (¡NUEVO!)
for i in range(N_SAMPLES):
    # Obtener la imagen, quitando la dimensión de canal 1
    image_data = reconstructed_images[i, :, :, 0]

    # Matplotlib puede guardar el array numpy directamente
    plt.imsave(
        os.path.join(OUTPUT_PNG_DIR, f"letter_{TEST_LETTER}_{i:02d}.png"),
        image_data,
        cmap='gray' # Es crucial especificar 'gray' para el formato gris
    )

# 5. Visualización (Opcional, pero útil para ver el resultado)
fig = plt.figure(figsize=(12, 3))
plt.suptitle(f"Generación y Guardado de {N_SAMPLES} PNGs", fontsize=16)

for i in range(N_SAMPLES):
    ax = fig.add_subplot(1, N_SAMPLES, i + 1)
    ax.imshow(reconstructed_images[i, :, :, 0], cmap='gray')
    ax.axis("off")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

print(f"✅ Todas las imágenes fueron guardadas en: {OUTPUT_PNG_DIR}")